<a href="https://colab.research.google.com/github/gusat/t0/blob/master/smd_v1x_3_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Part 1: Imports and Installations

import os
import ast
import csv
import sys
import json
from pickle import dump
import tensorflow as tf
print(tf.__version__)
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import SpectralClustering
from sklearn.metrics import calinski_harabasz_score
import more_itertools as mit
from sklearn.metrics import davies_bouldin_score

# Install Box for dictionaries with attribute access
!pip install python-box
from box import Box
# Clean up the prior installs
#!rm -r /kaggle/working/tmp

# # NeuroAI install: clone to a directory that won't match the name 'neuroaikit', e.g. tmp
# !git clone https://github.com/IBM/neuroaikit.git tmp
# !pip install --editable tmp
# # Manually add a search path: Python will fail to match 'neuroaikit' and then search under this path and find it:
# import sys
# sys.path.append('/kaggle/working/tmp')

# import neuroaikit as ai
# print(dir(ai))

# import neuroaikit.tf as aitf
# print(dir(aitf))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2.13.0
  Obtaining dependency information for python-box from https://files.pythonhosted.org/packages/65/63/c2df733c7e9549c1acc6919a3157f8f2fe833fa013052b169e228e1aeb75/python_box-7.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 10.0 MB/s eta 0:00:0000:0100:01


In [ ]:
# # Test SNU import
# neuron = aitf.layers.SNU(1)
# neuron # => <keras.layers.rnn.base_rnn.RNN at 0x7aa438a8a350> is OK

In [ ]:
# Part 2: Key post-proc. helpers (Telemanom et al. filters) for AE-based AD
# Helper classes and functions using the AE reconstruction error as input signal (named Errors, err_... etc.)

class Errors:
    def __init__(self, error_agg, error, real_data):
        """
        Batch processing of errors between actual and predicted values
        for a given error signal.
        Args:
            error_agg (ndarray): Exponentially-smoothed prediction error
            error (ndarray): Errors in prediction (predicted - actual)
            real_data (ndarray): Actual data
        Attributes:
            window_length (int): Number of trailing batches to use in error calculation
            nb_windows (int): Number of windows in test values for channel
            i_anom (arr): Indices of anomalies in channel test values
            E_seq (arr of tuples): Array of (start, end) indices for each continuous anomaly sequence
            anom_scores (arr): Score indicating relative severity of each anomaly sequence in E_seq
            e (arr): Errors in prediction (predicted - actual)
            e_s (arr): Exponentially-smoothed errors in prediction
        """
        self.window_length = 2016
        self.nb_windows = 4
        self.stride = 144

        self.i_anom = np.array([])
        self.E_seq = []
        self.anom_scores = []

        self.e_s = error_agg  # smoothed prediction error
        self.e_full = error
        self.real_data = real_data


    def merge_scores(self):
        """
        If anomalous sequences from subsequent batches are adjacent they
        will automatically be combined. This combines the scores for these
        initial adjacent sequences (scores are calculated as each batch is
        processed) where applicable.
        """
        merged_scores = []
        score_end_indices = []

        for i, score in enumerate(self.anom_scores):
            if not score["start_idx"] - 1 in score_end_indices:
                merged_scores.append(score["score"])
                score_end_indices.append(score["end_idx"])


    def process_batches(self):
        """
        Top-level function for the Error class that loops through batches
        of values for a given error signal.
        Args:
            error (ndarray): 1D error signal
        """

        i = 0
        break_flag = False
        idx_start = 0
        idx_end = idx_start + (self.window_length * self.nb_windows)

        while True:

            if idx_end > self.e_s.shape[0]:
                idx_end = self.e_s.shape[0]
                idx_start = idx_end - (self.window_length * self.nb_windows)
                break_flag = True
            # print("-> Evaluating", self.group_name, ":", round(idx_end / self.e_s.shape[0] * 100, 2), "%", end='\r')

            while True:

                # Adapt start idx based on present data gaps and anomaly windows
                nb_data_gaps = (
                    np.sum(self.real_data[idx_start:idx_end], axis=1) == 0
                ).sum()
                nb_i_anom = len(
                    [x for x in self.i_anom if idx_start <= x and x < idx_end]
                )
                idx_start = idx_start - (nb_data_gaps + nb_i_anom)
                idx_start = max(0, idx_start)

                # Compute updated window length
                nb_data_gaps = (
                    np.sum(self.real_data[idx_start:idx_end], axis=1) == 0
                ).sum()
                nb_i_anom = len(
                    [x for x in self.i_anom if idx_start <= x and x < idx_end]
                )
                len_cur_window = (idx_end - idx_start) - (nb_data_gaps + nb_i_anom)

                # Break if min window length
                condition1 = (self.window_length * self.nb_windows) <= len_cur_window
                condition2 = idx_start == 0
                if condition1 or condition2:
                    break

            window = ErrorWindow(
             idx_start, idx_end, self, i
            )
            window.find_epsilon()
            # window.find_epsilon(inverse=True)
            window.compare_to_epsilon()
            # window.compare_to_epsilon(inverse=True)

            if len(window.i_anom) == 0 and len(window.i_anom_inv) == 0:
                if break_flag == True:
                    break
                i = i + 1
                idx_start = i * self.stride
                idx_end = idx_start + (self.window_length * self.nb_windows)
                continue

            window.aw_fp_mitigation()
                # window.prune_anoms(inverse=True)

            if len(window.i_anom) == 0 and len(window.i_anom_inv) == 0:
                if break_flag == True:
                    break
                i = i + 1
                idx_start = i * self.stride
                idx_end = idx_start + (self.window_length * self.nb_windows)
                continue

            window.i_anom = np.sort(
                np.unique(np.append(window.i_anom, window.i_anom_inv))
            ).astype("int")
            window.score_anomalies(idx_start)

            # Update indices to reflect true indices in full set of values
            self.i_anom = np.sort(
                np.unique(np.append(self.i_anom, window.i_anom + idx_start))
            )
            self.anom_scores = self.anom_scores + window.anom_scores

            # Update loop variables
            if break_flag:
                break

            i = i + 1
            idx_start = i * self.stride
            idx_end = idx_start + (self.window_length * self.nb_windows)

        if len(self.i_anom) > 0:
            # Group anomalous indices into continuous sequences
            groups = [list(group) for group in mit.consecutive_groups(self.i_anom)]
            self.E_seq = [(int(g[0]), int(g[-1])) for g in groups if not g[0] == g[-1]]
            # aw_vis(self) #TODO: Added for test validation only
            self.merge_scores()


class ErrorWindow:
    def __init__(self, idx_start, idx_end, errors, idx_stride):
        """
        Represents a window for processing errors within a specified index range.
        Args:
            idx_start (int): Starting index of the window
            idx_end (int): Ending index of the window
            errors (Errors): Errors class object containing the error signal
            idx_stride (int): Index stride for the window
        """

        self.idx_start = idx_start
        self.idx_end = idx_end

        self.idx_stride = idx_stride

        self.prev_i_anom = [
            (int(x) - idx_start)
            for x in errors.i_anom
            if idx_start <= x and x <= idx_end
        ]

        self.i_anom = np.array([])
        self.E_seq = np.array([])
        self.non_anom_max = -1000000
        self.i_anom_inv = np.array([])
        self.E_seq_inv = np.array([])
        self.non_anom_max_inv = -1000000

        # TODO: self.error_buffer = self.config.as_threshold.rolling_window_length_mean
        self.error_buffer = 6
        self.magnitude_similarity_threshold = (
            0.2
        )
        self.duration_similarity_threshold = (
            0.2
        )
        self.sd_lim_low = 0
        self.sd_lim = 12

        self.sd_threshold = self.sd_lim
        self.sd_threshold_inv = self.sd_lim

        self.anom_scores = []
        self.e_s = errors.e_s[idx_start:idx_end].copy()
        self.e_full = errors.e_full[idx_start:idx_end].copy()
        self.real_data = errors.real_data[idx_start:idx_end].copy()

        self.mean_e_s = np.mean(self.e_s)
        self.sd_e_s = np.std(self.e_s)
        self.e_s_inv = np.array([self.mean_e_s + (self.mean_e_s - e) for e in self.e_s])

        self.epsilon = self.mean_e_s + self.sd_lim * self.sd_e_s
        self.epsilon_inv = self.mean_e_s + self.sd_lim * self.sd_e_s

        self.values = (
            errors.real_data[idx_start:idx_end]
            if len(errors.real_data.shape) == 1
            else (np.sum(errors.real_data[idx_start:idx_end], axis=1))
        )
        self.sd_values = np.std(self.values)

        self.perc_high, self.perc_low = np.percentile(self.values, [95, 5])
        self.inter_range = self.perc_high - self.perc_low


    def find_epsilon(self, inverse=False):
        """
        Find the anomaly threshold that maximizes function representing
        tradeoff between:
            a) number of anomalies and anomalous ranges
            b) the reduction in mean and st dev if anomalous points are removed
            from errors
        (see https://arxiv.org/pdf/1802.04431.pdf)
        Args:
            inverse (bool): If true, epsilon is calculated for inverted errors
        """
        # Mask error where data gaps appear or known anomalies exist
        mask_data_gap = np.sum(self.real_data, axis=1) == 0
        mask_i_anom = np.zeros(self.e_s.shape, dtype=bool)
        mask_i_anom[self.prev_i_anom] = True
        mask_threshold = mask_data_gap | mask_i_anom

        e_s = np.ma.masked_array(
            self.e_s if not inverse else self.e_s_inv, mask=mask_threshold
        )
        mean_e_s = e_s.mean()
        sd_e_s = e_s.std()

        max_score = -10000000

        list_epsilons = []
        for z in np.arange(self.sd_lim_low, self.sd_lim, 0.5):
            epsilon = mean_e_s + (sd_e_s * z)
            pruned_e_s = e_s[e_s < epsilon]

            i_anom = np.argwhere(e_s >= epsilon).reshape(
                -1,
            )
            buffer = np.arange(1, self.error_buffer)
            i_anom = np.sort(
                np.concatenate(
                    (
                        i_anom,
                        np.array([i + buffer for i in i_anom]).flatten(),
                        np.array([i - buffer for i in i_anom]).flatten(),
                    )
                )
            )
            i_anom = i_anom[(i_anom < len(e_s)) & (i_anom >= 0)]
            i_anom = np.sort(np.unique(i_anom))

            if len(i_anom) > 0:
                # Group anomalous indices into continuous sequences
                groups = [list(group) for group in mit.consecutive_groups(i_anom)]
                E_seq = [(g[0], g[-1]) for g in groups if not g[0] == g[-1]]

                mean_perc_decrease = (mean_e_s - pruned_e_s.mean()) / mean_e_s
                sd_perc_decrease = (sd_e_s - pruned_e_s.std()) / sd_e_s
                score = (mean_perc_decrease + sd_perc_decrease) / (
                    len(E_seq) ** 2 + len(i_anom)
                )  # original

                list_epsilons.append((z, score))

                # Sanity checks / guardrails to update threshold level
                # - current threshold level must produce less then 5 anomaly sequences
                # - current threshold level must threhold less then 50% of data points as anomalous

                if (
                    score >= max_score
                    and len(E_seq) <= 5  # TODO take away
                    and len(i_anom) < (len(e_s) * 0.5)
                ):
                    max_score = score
                    if not inverse:
                        self.sd_threshold = z
                        self.epsilon = mean_e_s + z * sd_e_s
                    else:
                        self.sd_threshold_inv = z
                        self.epsilon_inv = mean_e_s + z * sd_e_s

        # Plot epsilon score for current window
        # plot_telemanom_epsilon(
        #     config=self.config,aw
        #     group_name=self.group_name,
        #     idx_start=self.idx_start,
        #     idx_end=self.idx_end,
        #     signal=e_s,
        #     epsilon=self.epsilon,
        #     epsilon_scores=list_epsilons)


    def compare_to_epsilon(self, inverse=False):
        """
        Compare smoothed error values to epsilon (error threshold) and group
        consecutive errors together into sequences.
        Args:
            inverse (bool): If true, epsilon is calculated for inverted errors
            errors_all (obj): Errors class object containing list of all
            previously identified anomalies in test set
        """
        e_s = self.e_s if not inverse else self.e_s_inv
        epsilon = self.epsilon if not inverse else self.epsilon_inv

        # also take the below away: TODO
        if (
            not (
                self.sd_e_s > (0.05 * self.sd_values)
                or max(self.e_s) > (0.05 * self.inter_range)
            )
            or not max(self.e_s) > 0.05
        ):
            return

        i_anom = np.argwhere(
            (e_s >= epsilon) & (e_s > 0.05 * self.inter_range)
        ).reshape(
            -1,
        )

        if len(i_anom) == 0:
            return

        buffer = np.arange(1, self.error_buffer + 1)

        i_anom = np.sort(
            np.concatenate(
                (
                    i_anom,
                    np.array([i + buffer for i in i_anom]).flatten(),
                    np.array([i - buffer for i in i_anom]).flatten(),
                )
            )
        )
        i_anom = i_anom[(i_anom < len(e_s)) & (i_anom >= 0)]
        i_anom = np.sort(np.unique(i_anom))

        # Capture max of non-anomalous values below the threshold
        # (used in filtering process)
        window_indices = np.arange(0, len(e_s))
        candidate_indices = np.setdiff1d(window_indices, i_anom)
        non_anom_max = np.max(np.take(e_s, candidate_indices))

        # Group anomalous indices into continuous sequences
        groups = [list(group) for group in mit.consecutive_groups(i_anom)]
        E_seq = [(g[0], g[-1]) for g in groups if not g[0] == g[-1]]

        if inverse:
            self.i_anom_inv = i_anom
            self.E_seq_inv = E_seq
            self.non_anom_max_inv = non_anom_max
        else:
            self.i_anom = i_anom
            self.E_seq = E_seq
            self.non_anom_max = non_anom_max


    def score_anomalies(self, idx_start):
        """
        Calculate anomaly scores based on max distance from epsilon
        for each anomalous sequence.
        Args:
            prior_idx (int): starting index of window within full set of test
                values for channel
        """

        groups = [list(group) for group in mit.consecutive_groups(self.i_anom)]

        for e_seq in groups:

            score_dict = {
                "start_idx": e_seq[0] + idx_start,
                "end_idx": e_seq[-1] + idx_start,
                "score": 0,
            }

            score = max(
                [
                    abs(self.e_s[i] - self.epsilon) / (self.mean_e_s + self.sd_e_s)
                    for i in range(e_seq[0], e_seq[-1] + 1)
                ]
            )
            inv_score = max(
                [
                    abs(self.e_s_inv[i] - self.epsilon_inv)
                    / (self.mean_e_s + self.sd_e_s)
                    for i in range(e_seq[0], e_seq[-1] + 1)
                ]
            )
            # The max score indicates whether anomaly was from regular
            # or inverted errors
            score_dict["score"] = max([score, inv_score])
            self.anom_scores.append(score_dict)


    def aw_fp_mitigation(self, inverse=False):
        """
        Remove anomalies that don't meet minimum separation from the next
        closest anomaly or error value
        Args:
            inverse (bool): If true, epsilon is calculated for inverted errors
        """
        E_seq = self.E_seq if not inverse else self.E_seq_inv
        e_s = self.e_s if not inverse else self.e_s_inv
        non_anom_max = self.non_anom_max if not inverse else self.non_anom_max_inv

        if len(E_seq) == 0:
            return

        # (1) Self-similarity space: Novelty pruning based on similar magnitude. Remove anomalies of similar magnitude.
        # Assumption: Anomalies of same magnitued are not frequently recurring within the same channel
        E_seq_max = np.array([max(e_s[e[0]:e[-1]+1]) for e in E_seq])
        E_seq_max_sorted = np.sort(E_seq_max)[::-1]
        E_seq_max_sorted = np.append(E_seq_max_sorted, [non_anom_max])

        i_to_remove = np.array([], dtype=int)
        for i in range(0, len(E_seq_max_sorted)-1):
            cur_similarity = abs(E_seq_max_sorted[i] - E_seq_max_sorted[i+1]) / (E_seq_max_sorted[i] + E_seq_max_sorted[i+1])
            # print(f"-> AW {i} | Severity similarity: {cur_similarity}")
            if cur_similarity < self.magnitude_similarity_threshold:
                i_to_remove = np.append(i_to_remove, np.argwhere(E_seq_max == E_seq_max_sorted[i]))
                i_to_remove = np.append(i_to_remove, np.argwhere(E_seq_max == E_seq_max_sorted[i+1]))
                # print("-> Change of magnitude low:", round(abs(E_seq_max_sorted[i] - E_seq_max_sorted[i+1]) / E_seq_max_sorted[i], 2))

        i_to_remove = np.unique(i_to_remove)
        i_to_remove.sort()

        # print("-> Number of 1st stage removal candidates", len(i_to_remove))

        # (2) Self-similarity time: Pruning of candidates based on similar duration. Remove anomalies of similar magnitude and duration.
        # Assumption: Anomalies of same magnitude and duration are not frequently recurring within the same
        if len(i_to_remove) > 1:

            E_seq_lengths = np.array([ e[1]-e[0] for e in np.asarray(E_seq)[i_to_remove] ])
            E_seq_lengths_sorted = np.sort(np.unique(E_seq_lengths))[::-1]

            if not len(E_seq_lengths_sorted) == 1:
                E_seq_lengths_sorted = np.append(E_seq_lengths_sorted, [E_seq_lengths_sorted[-2]])
                i_to_remove_2nd = np.array([], dtype=int)

                for i in range(0, len(E_seq_lengths_sorted)-1):
                    if ((abs(E_seq_lengths_sorted[i] - E_seq_lengths_sorted[i+1]) / (E_seq_lengths_sorted[i] + E_seq_lengths_sorted[i+1])) < self.duration_similarity_threshold):
                        i_to_remove_2nd = np.append(i_to_remove_2nd, np.argwhere(E_seq_lengths == E_seq_lengths_sorted[i]))
                        i_to_remove_2nd = np.append(i_to_remove_2nd, np.argwhere(E_seq_lengths == E_seq_lengths_sorted[i+1]))
                        # print("-> Change of duration low:", round(abs(E_seq_lengths_sorted[i] - E_seq_lengths_sorted[i+1]) / E_seq_lengths_sorted[i], 2))

                i_to_remove_2nd = np.unique(i_to_remove_2nd)
                i_to_remove_2nd.sort()

                # print("-> Number of 2st stage removal candidates", len(i_to_remove_2nd))

                if len(i_to_remove_2nd) > 0:
                    i_to_remove = i_to_remove[i_to_remove_2nd]
                else:
                    i_to_remove = np.array([], dtype=int)
        if len(i_to_remove) > 0:
            # print("-> Removing", len(i_to_remove), "novelties due to similar magnitude /and duration")
            E_seq = np.delete(E_seq, i_to_remove, axis=0)

        if len(E_seq) == 0 and inverse:
            self.i_anom_inv = np.array([])
            return
        if len(E_seq) == 0 and not inverse:
            self.i_anom = np.array([])
            return

        indices_to_keep = np.concatenate([range(e_seq[0], e_seq[-1]+1) for e_seq in E_seq])

        if not inverse:
            mask = np.isin(self.i_anom, indices_to_keep)
            # Count true values before vs after
            self.i_anom = self.i_anom[mask]
            self.E_seq = E_seq

        else:
            mask_inv = np.isin(self.i_anom_inv, indices_to_keep)
            self.i_anom_inv = self.i_anom_inv[mask_inv]
            self.E_seq_inv = E_seq


In [ ]:

from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from keras_nlp.layers import TransformerEncoder, TransformerDecoder, SinePositionEncoding
#import aitf  # Import the NeuroAI toolkit

# Part 3a: Define the 4 core AE models for AD: SNU- / LSTM- / 2x Transformer-based Autoencoders

# Define LSTM Model
def build_lstm_model():
    """Instantiates the lstm / AE architecture."""
    # Initialize early stopping callback and optimizer
    early_stop = EarlyStopping(
        monitor="val_loss", patience=5, verbose=1, restore_best_weights=True
    )
    optimizer = Adam(
        learning_rate=0.1,
        clipnorm=1,
    )

    # Create model
    model = models.Sequential()
    model.add(layers.Input(shape=(config.window_length, config.num_features)))
    model.add(
        layers.LSTM(
            128,
            return_sequences=True,
            input_shape=(config.window_length, config.num_features),
        )
    )
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.LSTM(7, return_sequences=True))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.LSTM(config.num_features, return_sequences=True))
    model.summary()

    # Build and compile model
    print("Compiling model...")
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=["mse", "mae"],
    )

    return model, early_stop

# Define Transformer Model
def build_transformer_model():
    early_stop = keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5, verbose=10, restore_best_weights=True
    )
    optimizer = Adam(
        learning_rate=0.0005,
        clipnorm=1,
    )
    # add a time encoding:

    inputs = keras.Input(shape=(config.window_length, config.num_features))

    x = inputs
    positional_encoding = SinePositionEncoding()(x)
    x = x + positional_encoding
    for idx in range(2):
        x = TransformerEncoder(
            intermediate_dim=128,
            num_heads=5,
            dropout=0.1,
        )(x)
        x = layers.Dense([20,7][idx],
                         activation='relu')(x)

    # x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    # decoder: (usinfg encoder here actually as we do not want masking)
    for idx, _ in enumerate(
        range(2)
    ):
        x = TransformerEncoder(
            intermediate_dim=128,
            num_heads=5,
            dropout=0.1,
        )(x)
        x = layers.Dense(
            [20, 32][idx],
            activation="relu",
        )(x)

    # for dim in config.AD_model.TransformerAE.mlp_units:
    # x = layers.Dense(dim, activation="relu")(x)
    # x = layers.Dropout(config.AD_model.TransformerAE.mlp_dropout)(x)

    # decoder between the last two

    outputs = layers.Dense(38, activation='relu')(x)
    model = keras.Model(inputs, outputs)
    model.summary()

    print("Compiling model...")
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=["mse", "mae"],
    )
    return model, early_stop


# Part 3b: SNU-based LSTM and transformer AE models

# Define the configuration for the SNU layers
config = {'activation': tf.nn.sigmoid, 'recurrent': True, 'decay': 0.98, 'g': tf.identity}
# config = {'activation': tf.nn.leaky_relu, 'recurrent': True, 'decay': 0.98, 'g': tf.identity} # NaN in fit
input_shape = (120, 38) # used hard-coded for SNU models

def build_snu_model():
    """Instantiates the SNU-based / LSTM-like AE architecture."""
    # Initialize early stopping callback and optimizer
    early_stop = EarlyStopping(
        monitor="val_loss", patience=5, verbose=1, restore_best_weights=True
    )
    optimizer = Adam(
        learning_rate=0.001,
        clipnorm=0.5,
    )

    # Create model
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(
        aitf.layers.SNU(
            128,
            **config,
            return_sequences=True
        )
    )

    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(rate=0.2))
    model.add(aitf.layers.SNU(7, **config, return_sequences=True))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(rate=0.2))
    model.add(aitf.layers.SNU(38, **config, return_sequences=True))
    model.summary()

    # Build and compile model
    print("Compiling model...")
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=["mse", "mae"],
    )

    return model, early_stop


# B) Simple SNU-based transformer AE
def build_snu_transformer_AE():
    """Instantiates the SNU / Transformer AE architecture.
    # Arguments
        config: configuration file containing hyperparameters to build AE
    # Returns
        A Keras model instance 88k (vs. 34k native) parms
    """
    # Initialize early stopping callback and optimizer
    early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5, verbose=10, restore_best_weights=True
    )
    optimizer = Adam(
        learning_rate=0.0005,
        clipnorm=1,
    )

    # Define the configuration for the SNU layers
    config_snu = {'activation': tf.nn.sigmoid, 'recurrent': True, 'decay': 0.995, 'g':  tf.identity}

    # Create model
    inputs = keras.Input(shape=input_shape)
    x = inputs
    positional_encoding = SinePositionEncoding()(x)
    x = x + positional_encoding
    # positional_encoding = aitf.layers.SinePositionEncoding()(x) => functionally equiv., more explicit and flexible method to add layer
    # x = layers.Add()([x, positional_encoding])

    for idx in range(2):
        x = aitf.layers.SNU(
            128,
            **config_snu,
            return_sequences=True
        )(x)
        x = layers.Dense([20,7][idx], activation='relu')(x)

    # Decoder
    for idx, _ in enumerate(range(2)):
        x = aitf.layers.SNU(
            128,
            **config_snu,
            return_sequences=True
        )(x)
        x = layers.Dense([20, 32][idx], activation="relu")(x)

    outputs = layers.Dense(38, activation='relu')(x)
    model = keras.Model(inputs, outputs)
    model.summary()

    # Build and compile model
    print("Compiling model...")
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=["mse", "mae"],
    )

    return model, early_stopping

# Example usage SNU models:
# input_shape = (sequence_length, features)  # Replace with our actual input shape (120,38)
# model, early_stopping = build_snu_transformer_AE(input_shape)

# Build an AE model of the 4 possible autoencoders (2x plain + 2 SNU-based versions)
# model, early_stop = build_snu_transformer_AE()
# model1, early_stop1 = build_snu_model()  # => LSTM-like SNU-based AE
# model2, early_stop2 = build_lstm_model()
# model3, early_stop3 = build_transformer_model()

Using TensorFlow backend


In [ ]:
# Part 4a: SMD Corpus FE / Dataset cooking...
# Dataset Preparation and Ingestion
#=================================================================
# Prepare the SMD dataset for ingestion
# IN folder from the default SMD dataset created in kaggle
dataset_path = '/kaggle/input'
print("kaggle SMD dataset input from = ", dataset_path)

# OUT folder for pre-processing the SMD raw dataset
output = '/kaggle/working'
output_folder = os.path.join(output, 'ServerMachineDataset', 'processed')
print("output_folder = ", output_folder)

# %%
# IO helper functions
os.makedirs(output_folder, exist_ok=True)

def load_and_save(category, filename, dataset, dataset_folder):
    temp = np.genfromtxt(os.path.join(dataset_path,dataset_folder, category, filename),
                         dtype=np.float32,
                         delimiter=',')
    print(dataset, category, filename, temp.shape)
    with open(os.path.join(output_folder, dataset + "_" + category + ".pkl"), "wb") as file:
        dump(temp, file)

def load_data(dataset):
    if dataset == 'SMD':
        dataset_folder = 'smd-onmiad/ServerMachineDataset'
        file_list = os.listdir(os.path.join(dataset_path, dataset_folder, "train"))
        for filename in file_list:
            if filename.endswith('.txt'):
                load_and_save('train', filename, filename.strip('.txt'), dataset_folder)
                load_and_save('test', filename, filename.strip('.txt'), dataset_folder)
                load_and_save('test_label', filename, filename.strip('.txt'), dataset_folder)

def list_folder(folder):
    for dirname, _, filenames in os.walk(folder):
        for filename in filenames:
            print(os.path.join(dirname, filename))


# %%
# Prepare the SMD dataset: Contains train and test data from 28 SMD servers, each 38 KPIs (anon, i.e., names are masked), for 5 weeks, Tsampling ~ 2min
# add here
import pickle
from sklearn.preprocessing import MinMaxScaler
def get_data(dataset, max_train_size=None, max_test_size=None, print_log=True, do_preprocess=True, train_start=0,
             test_start=0):
    """
    get data from pkl files
    return shape: (([train_size, x_dim], [train_size] or None), ([test_size, x_dim], [test_size]))
    """
    if max_train_size is None:
        train_end = None
    else:
        train_end = train_start + max_train_size
    if max_test_size is None:
        test_end = None
    else:
        test_end = test_start + max_test_size
    print('load data of:', dataset)
    print("train: ", train_start, train_end)
    print("test: ", test_start, test_end)
    x_dim = 38
    f = open(os.path.join(output_folder, dataset + '_train.pkl'), "rb")
    train_data = pickle.load(f).reshape((-1, x_dim))[train_start:train_end, :]
    f.close()
    try:
        f = open(os.path.join(output_folder, dataset + '_test.pkl'), "rb")
        test_data = pickle.load(f).reshape((-1, x_dim))[test_start:test_end, :]
        f.close()
    except (KeyError, FileNotFoundError):
        test_data = None
    try:
        f = open(os.path.join(output_folder, dataset + "_test_label.pkl"), "rb")
        test_label = pickle.load(f).reshape((-1))[test_start:test_end]
        f.close()
    except (KeyError, FileNotFoundError):
        test_label = None
    if do_preprocess:
        train_data = preprocess(train_data)
        test_data = preprocess(test_data)
    print("train set shape: ", train_data.shape)
    print("test set shape: ", test_data.shape)
    print("test set label shape: ", test_label.shape)
    return (train_data, None), (test_data, test_label)


def preprocess(df):
    """returns normalized and standardized data.
    """

    df = np.asarray(df, dtype=np.float32)

    if len(df.shape) == 1:
        raise ValueError('Data must be a 2-D array')

    if np.any(sum(np.isnan(df)) != 0):
        print('Data contains NaN values. Will be replaced with 0')
        df = np.nan_to_num()

#     # normalize data -> SMD data is natively normalized already [0,1]
#     df = MinMaxScaler().fit_transform(df)
#     print('Data normalized')

    return df

kaggle SMD dataset input from =  /kaggle/input
output_folder =  /kaggle/working/ServerMachineDataset/processed


In [ ]:
# Load SMD
load_data('SMD')
#list_folder('/kaggle/working')
print("Done loading and pre-processing SMD into pkl files.")

machine-3-1 train machine-3-1.txt (28700, 38)
machine-3-1 test machine-3-1.txt (28700, 38)
machine-3-1 test_label machine-3-1.txt (28700,)
machine-1-1 train machine-1-1.txt (28479, 38)
machine-1-1 test machine-1-1.txt (28479, 38)
machine-1-1 test_label machine-1-1.txt (28479,)
machine-3-5 train machine-3-5.txt (23690, 38)
machine-3-5 test machine-3-5.txt (23691, 38)
machine-3-5 test_label machine-3-5.txt (23691,)
machine-1-5 train machine-1-5.txt (23705, 38)
machine-1-5 test machine-1-5.txt (23706, 38)
machine-1-5 test_label machine-1-5.txt (23706,)
machine-2-9 train machine-2-9.txt (28722, 38)
machine-2-9 test machine-2-9.txt (28722, 38)
machine-2-9 test_label machine-2-9.txt (28722,)
machine-2-8 train machine-2-8.txt (23702, 38)
machine-2-8 test machine-2-8.txt (23703, 38)
machine-2-8 test_label machine-2-8.txt (23703,)
machine-3-11 train machine-3-11.txt (28695, 38)
machine-3-11 test machine-3-11.txt (28696, 38)
machine-3-11 test_label machine-3-11.txt (28696,)
machine-3-7 train mac

In [ ]:
import os
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import time # for training stats

# Class to measure the time taken for each epoch during training / fine stats
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

# Create a general Output directory to store results
results_dir = '/kaggle/working/results2/'
os.makedirs(results_dir, exist_ok=True)

# Initialize a list to store the history objects and training times per model (external to the loops)
# Store the total training time and history object
histories = []
total_training_times = [] #aggregated stat, coarse
performance_metrics = []

# List of model names
# model_names = ["snu-ae", "snu-transformer", "lstm", "transformer"]
model_names = ["lstm", "transformer"]

# Define a dictionary for model indices used for Loop#2; default range(4), or less for quick trials
model_indices = {
#     'snu-ae': 1,
#     'snu-transformer': 2,
    'lstm': 1,
    'transformer': 2
}

# Initialize a list to store the models
model_list = []

# Initialize the TimeHistory callback, fine (external to the loops)
time_callback = TimeHistory()

# Define our callbacks common to all models (external to the loops)
early_stop = EarlyStopping(monitor="val_loss", patience=5, verbose=1, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=7, min_lr=0.00001)

# Define and build our 4 (or less) AE models
# model_builders = [build_snu_model, build_snu_transformer_AE, build_lstm_model, build_transformer_model]
model_builders = [build_lstm_model, build_transformer_model]

# Configs build: Loop over each model to configure correctly
for Mod_i in range(2):
    # Set the config dictionary based on the type of the model
    if (model_builders[Mod_i] == build_snu_model) or (model_builders[Mod_i] == build_snu_transformer_AE):
        config = {'activation': tf.nn.sigmoid, 'recurrent': True, 'decay': 0.98, 'g': tf.identity} #input_shape = (sequence_length, features) => (120,38)
    else:
        config =  Box({"window_length":120, "num_features":38}) # used for the plain, non-SNU models

    model, early_stop = model_builders[Mod_i]()
    callbacks = [early_stop, lr_scheduler, time_callback]

    # Add the model to the model list
    model_list.append(model)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 120, 128)          85504     
                                                                 
 batch_normalization (Batch  (None, 120, 128)          512       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 120, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 120, 7)            3808      
                                                                 
 batch_normalization_1 (Bat  (None, 120, 7)            28        
 chNormalization)                                                
                                                                 
 dropout_1 (Dropout)         (None, 120, 7)            0

In [ ]:
# Load test labels (assuming a 1D numpy array of binary values)
test_labels = np.loadtxt("/kaggle/input/smd-onmiad/ServerMachineDataset/test_label/machine-1-7.txt")

label_indexes = []
start_timestamps = []
end_timestamps = []

# Keep track of the current start and previous label
current_start = 0
prev_label = None

# Keep track of the current label segment and its index
current_label = None
segment_index = 0

# Iterate through each label value
for i, label in enumerate(test_labels):
    # Check for label change or end of data
    if label != prev_label or i == len(test_labels) - 1:
        # Add information for the previous segment (if it was a 1 segment)
        if prev_label == 1:
            label_indexes.append(segment_index)  # Use segment_index directly
            start_timestamps.append(current_start)
            end_timestamps.append(i - 1)  # Adjust for zero-based indexing

        # Start a new segment for the current label
        current_start = i
        prev_label = label
        segment_index += 1 if label == 1 else 0  # Increment only for 1 segments

# Print or store the extracted information
print("Label indexes:", label_indexes)
print("Start timestamps:", start_timestamps)
print("End timestamps:", end_timestamps)

Label indexes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
Start timestamps: [837, 2959, 5849, 6031, 7099, 8564, 12359, 13799, 15239, 15960, 18109, 19442, 19559]
End timestamps: [857, 4173, 5939, 6033, 7999, 8579, 12373, 13824, 15269, 15962, 18149, 19445, 19589]


## XAD = eXplainable AD, starting w/ the top culprit KPIs, to be extracted per event window
### Currently under trials, with unstable XAD snippets for top contribs (challenges in cleaning up the code for culprit KPI identinfication per AD event, not mere avgs.)

In [ ]:
# Main / loop#1: All 28 machine names from the SMD dataset
# machine_names = ["machine-1-1", "machine-1-2", "machine-1-3", "machine-1-4", "machine-1-5", "machine-1-6", "machine-1-7", "machine-1-8", "machine-2-1", "machine-2-2", "machine-2-3", "machine-2-4", "machine-2-5", "machine-2-6", "machine-2-7", "machine-2-8", "machine-2-9", "machine-3-1", "machine-3-2", "machine-3-3", "machine-3-4", "machine-3-5", "machine-3-6", "machine-3-7", "machine-3-8", "machine-3-9", "machine-3-10", "machine-3-11"]

machine_names = ["machine-1-7"]
# Loop over each machine
for i_mach, machine_name in enumerate(machine_names):
    dataset = f"{machine_name}"

    (x_train, _), (x_test, y_test) = \
            get_data(dataset, None, None, train_start=0,
                      test_start=0)

    # Part 4c: Train & Test preps
    # Build the SMD training dataset In == Out tensor shape for AE, batches / wndw, seq_size, stride chosen as in the SI wild dataset (SMD = 1min/step; IBM SI = 5min/step)
    ds_train = tf.keras.utils.timeseries_dataset_from_array(
                    data=x_train,
                    targets=None,
                    sequence_length=120,
                    sequence_stride=1,
                    sampling_rate=1,
                    batch_size=1024,
                    shuffle=False
                )
    ds_train = tf.data.Dataset.zip((ds_train, ds_train))

    # Validation set build: skip stride=seq
    ds_validation = tf.keras.utils.timeseries_dataset_from_array(
                    data=x_test,
                    targets=None,
                    sequence_length=120,
                    sequence_stride=120,
                    sampling_rate=1,
                    batch_size=1024,
                    shuffle=False
                )
    ds_validation = tf.data.Dataset.zip((ds_validation, ds_validation))

    # Define the base directory
    base_dir = os.path.join(results_dir, machine_name)


    # Loop#2 over each model from our 4 model list to fit them (strings needed for IO, but keras objects for work)
    for Mod_i, model_name_str in enumerate(model_names):
        # Second main loop#2
        model = model_list[Mod_i]

        # Start the timer
        start_time = time.time()

        # Fit the model
        history = model.fit(
            x=ds_train,
            validation_data=ds_validation,
            epochs=25,
            callbacks=callbacks,
            workers=4,
            max_queue_size=10,
            batch_size=1024,
            shuffle=True
        )

        # Stop the timer and calculate the total training time per model
        end_time = time.time()
        total_training_time = end_time - start_time

        # Store the total training time and history object
        total_training_times.append(total_training_time)
        histories.append(history)

        # Print the total training time and average time per epoch for the current model
        print(f"Total training time for Model {Mod_i+1}: {total_training_time} seconds")
        print(f"Average time per epoch: {total_training_time/len(history.epoch)} seconds")


        machine_model_dir = os.path.join(base_dir, model_name_str)
        os.makedirs(machine_model_dir, exist_ok=True)
        print("Working dirs=", results_dir, "base_dir=", base_dir, "machine_name=", machine_name, "model_name_str=", model_name_str, "machine_model_dir=", machine_model_dir)

        # Define the metrics to plot
        metrics = ['loss', 'val_loss', 'mse', 'val_mse', 'mae', 'val_mae']

        # After fitting the model we manually extract its dictionary (could also use the defined metrics)
        #V history_data = history.history
        history_data = {
            'loss': history.history['loss'],
            'val_loss': history.history['val_loss'],
            'mse': history.history['mse'],
            'val_mse': history.history['val_mse'],
            'mae': history.history['mae'],
            'val_mae': history.history['val_mae']
        }

        # Now history_data is a dictionary that contains the loss and accuracy values for each epoch
        # We pickle this dictionary instead of the whole history object
        with open(os.path.join(machine_model_dir, f'training_data_model_{Mod_i+1}.pkl'), 'wb') as f:
            pickle.dump({'total_training_times': total_training_times, 'history_data': history_data}, f)

        #=========================================================
           # For each metric plot and save the training perf history
            for metric in metrics:
                plt.figure(figsize=(10, 5))
                plt.plot(history_data[metric], label=f'Training {metric}')
                if f'val_{metric}' in history_data:
                    plt.plot(history_data[f'val_{metric}'], label=f'Validation {metric}')
                plt.title(f'{metric.capitalize()} for Model {model_name_str} across all epochs')
                plt.xlabel('Epoch')
                plt.ylabel(metric.capitalize())
                plt.legend()
                plt.savefig(os.path.join(base_dir, f'{model_name_str}/{metric}_history_model_{Mod_i}.png'))
                plt.close()

            print("Training metrics saved for Server=", machine_name, "AE_model=", model_name_str, "Server&Model_dir=", machine_model_dir)

# Predict (AD) => Loop over each AE model in the model list to validate them vs. the test/validation set
# for Mod_i in range(4):
#     model = model_list[Mod_i]

            # Final Model resulting after training, is validated vs. the GT label dataset and perf. measured (Prec, Recall, F1 ...)
            # Prediction on validation set
            pred = model.predict(ds_validation)
            pred = np.vstack(
                [np.reshape(element, (-1, 38)) for element in pred]
            )

            # Ground truth label-based validation for individual machines (brute force approach)
            gt = np.vstack(
                [np.reshape(element, (-1, 38)) for element in ds_validation.map(lambda x, y: x)]
            )

            # Compute squared error of the model's prediction vs. the ground truth labeled SMD dataset
            err = np.square(gt - pred)
            cur_signal = err
            cur_data = gt
            print(np.shape(cur_signal))

            # Post-proc#1: Aggregate Reconstruction Errors across all KPIs and apply Telemanom optimizer for primary event detection
            cur_signal_agg = np.sum(cur_signal, axis=1)
            cur_signal_agg_ma = (
                pd.DataFrame(cur_signal_agg)
                .ewm(span=12)
                .mean()
                .values.flatten()
            )

            # Run Telemanom (TLM) for AD on the AE reconstruct_errors and obtain AD windows, aka. raw event intervals
            errors = Errors(cur_signal_agg_ma, cur_signal, cur_data)
            errors.process_batches()
            cur_idx_starts = [novelty_window[0] for novelty_window in errors.E_seq]
            cur_idx_ends = [novelty_window[-1] for novelty_window in errors.E_seq]

            # Print the event intervals
            print(cur_idx_starts)
            print(cur_idx_ends)

            # Plot binary array for anomaly intervals
            import numpy as np
            import matplotlib.pyplot as plt
            anomaly_intervals = np.zeros(max(cur_idx_ends) + 1)
            for start, end in zip(cur_idx_starts, cur_idx_ends):
                anomaly_intervals[start:end+1] = 1

            #print(f"Model {Mod_i+1} is {model.name}")
            plt.figure(figsize=(10, 3))
            plt.plot(anomaly_intervals)
            plt.title("Predicted Anomaly Intervals")
            plt.xlabel("Time Step")
            plt.ylabel("Anomaly")
            #plt.show()

            # Save the plot to a file
            plt.savefig(os.path.join(machine_model_dir, f'anomaly_intervals_model_{Mod_i+1}.png'))
            plt.close()

            # Calculate the raw/many AD event_durations before refinement in PPS#2 ?
            # Convert event window indexes into 0s for normal and 1s for raw AD events (raw: after only the primary detection AE+TLM)
            pred = np.zeros(np.shape(y_test))

            # To be done per event: Assuming event_duration is the average duration of our detected anomaly events
            # event_duration = np.mean(np.array(cur_idx_ends) - np.array(cur_idx_starts))
            # event_duration = int(event_duration)  # Convert to integer if needed
            event_duration = np.mean(np.array(cur_idx_ends) - np.array(cur_idx_starts)).astype(int)
            print("Average raw event duration:", event_duration)

            for i, start in enumerate(cur_idx_starts):
                end = cur_idx_ends[i] + 1
                pred[start:end] = 1

            # Calculate AE-based reconstruction errors
            rec_err = np.mean(err, axis=1)
            np.shape(rec_err)

            # Descriptive statistics of reconstruction errors
            pd.Series(rec_err).describe()

            # XAD defs
            # Generic KPI contributions functions to be recoded
            def calculate_kpi_contributions(models, data):
                """Calculate KPI contributions to reconstruction errors."""
                mse_list, kpi_contributions_list = [], []

                for model in models:
                    reconstructions = model.predict(data)
                    mse = np.mean(np.square(data - reconstructions), axis=(1, 2))
                    kpi_contributions = np.mean(np.abs(data - reconstructions), axis=(0, 1))

                    mse_list.append(mse)
                    kpi_contributions_list.append(kpi_contributions)

                return mse_list, kpi_contributions_list

            def accumulate_kpi_contributions(kpi_contributions_list, event_duration):
                """Accumulate KPI contributions over the entire event duration."""
                accumulated_contributions = np.zeros_like(kpi_contributions_list[0])
                min_length = min(event_duration, len(kpi_contributions_list)) # TBD per each event: event_duration = np.mean(np.array(cur_idx_ends) - np.array(cur_idx_starts)).astype(int)

                for i in range(min_length):
                    current_contributions = kpi_contributions_list[i]
                    accumulated_contributions += current_contributions
                    contributing_kpis = np.where(current_contributions > 0)[0] + 1  # Adding 1 to convert from zero-based to one-based index
#                     print(f"Step {i + 1} contributing KPIs:", contributing_kpis)
#                     for kpi_index in contributing_kpis:
#                         print(f"  KPI {kpi_index} contribution: {current_contributions[kpi_index - 1]}")
                print("Accumulated contribs/KPI:", accumulated_contributions)

                return accumulated_contributions


            # Assuming x_test has shape (~28k steps, 38 KPIs)
            desired_sequence_length = config['window_length']  # 120
            num_features = config['num_features']  # 38
            # Calculate the number of sequences we can create
            num_sequences = len(x_test) // desired_sequence_length
            # Calculate the number of elements to pad
            remaining_elements = len(x_test) % desired_sequence_length
            elements_to_pad = desired_sequence_length - remaining_elements
            # Pad the array with zeros
            x_test_padded = np.pad(x_test, ((0, elements_to_pad), (0, 0)), mode='constant')
            # Reshape the padded array
            x_test_reshaped = x_test_padded.reshape((num_sequences + 1, desired_sequence_length, num_features))

            # Extract top KPI contribs
            # Calculate KPI contributions for the SMD test dataset
            test_mse_list, test_kpi_contributions_list = calculate_kpi_contributions(model_list, x_test_reshaped)
            # x_test.reshape((-1, config['window_length'], config['num_features'])))

            # Access the top contributing KPIs during anomaly events of avg duration
            accumulated_contributions = accumulate_kpi_contributions(test_kpi_contributions_list, event_duration)
            top_contributors = np.argsort(accumulated_contributions)[::-1][:10]
            print("Culprit KPIs for all AD events of", machine_name, "Top-k=", top_contributors)


            # Compute AUROC, albeit less relevant here
            from sklearn.metrics import roc_auc_score
            roc_auc_score(y_test[:len(rec_err)], rec_err)

            # Set an heuristic/SME-based threshold for anomaly (pre-)classification
            thresh = 0.05
            label_pred = [1 if x >= thresh else 0 for x in rec_err]

            # Calculate the initial precision, recall, F1-score, and support;
            # also handling the "no predicted AD samples" rare case, which leads to zero_division
            P_bef, R_bef, F1_bef, _ = precision_recall_fscore_support(
                y_test[:len(label_pred)], label_pred, average="binary", zero_division=0
            )

            # Plot model's rec_err with a logarithmic scale on the Y-axis
            rec_err_normal = [x for i, x in enumerate(rec_err) if label_pred[i] == 0]
            rec_err_anomaly = [x for i, x in enumerate(rec_err) if label_pred[i] == 1]

            #print(f"Model {Mod_i+1} is {model.name}")
            plt.figure(figsize=(8, 4))
            plt.hist(rec_err_normal, bins=250, color='blue', alpha=0.95)
            plt.hist(rec_err_anomaly, bins=250, color='red', alpha=1)
            plt.yscale('log')
            plt.xlabel('Reconstruction Error')
            plt.ylabel('Frequency (log scale)')
            plt.title('Histogram of AE Model\'s Reconstruction Errors')
            plt.grid()
            #plt.show()
            plt.savefig(os.path.join(machine_model_dir, f'reconstruction_errors_histogram_model_{Mod_i+1}.png'))
            plt.close()

            # AD events detected before aggregation
            AD_events = np.sum(label_pred)
            print("AD_events detected =>", AD_events )


            # v3: Post-proc#2 (PPS2): AD windows' aggregation for bursty events
            def adjust_pred(test_true, test_pred):
              """
              Adjusts predicted anomaly labels based on true labels and event durations.

              Args:
                  test_true: A list of true anomaly labels (0: normal, 1: anomaly).
                  test_pred: A list of predicted anomaly labels (0: normal, 1: anomaly).

              Returns:
                  A tuple containing:
                      - Adjusted predicted anomaly labels (`test_pred_adjusted`).
                      - List of anomaly event start indices (`event_starts`).
                      - List of anomaly event end indices (`event_ends`).
              """

              # Initialize variables
              print("Evaluation: Adjusting predictions ...")
              anomaly_state = False
              event_starts = []
              event_ends = []

              # Iterate through each label pair
              for i, (true_label, pred_label) in enumerate(zip(test_true, test_pred)):
                # Check for start of an anomaly event
                if true_label == 1 and pred_label == 1 and not anomaly_state:
                  anomaly_state = True
                  event_starts.append(i)

                # Check for end of an anomaly event
                elif true_label == 0 and anomaly_state:
                  anomaly_state = False
                  event_ends.append(i - 1)

              # Handle potential last event extending to the end
              if anomaly_state:
                event_ends.append(len(test_true) - 1)

              # Adjust predictions based on event durations
              test_pred_adjusted = test_pred.copy()  # Avoid modifying original list
              for start, end in zip(event_starts, event_ends):
                # Extend anomaly prediction backward until true non-anomaly
                for j in range(start, 0, -1):
                  if test_true[j] == 0:
                    break
                  if test_pred_adjusted[j] == 0:
                    test_pred_adjusted[j] = 1

                # Extend anomaly prediction forward until true non-anomaly
                for j in range(end, len(test_true)):
                  if test_true[j] == 0:
                    break
                  if test_pred_adjusted[j] == 0:
                    test_pred_adjusted[j] = 1

              # Print informative message => GT_labels
              print("Adjustment completed; event starts", event_starts)
              print("Adjustment completed; event ends", event_ends)
              return test_pred_adjusted, event_starts, event_ends


            # AD events resulted after bursty AD post-proc#2, aggregated => predict events
            label_pred, event_starts, event_ends = adjust_pred(y_test[:len(label_pred)], label_pred)   #? => all the GT_labels
            AD_events_aggregated = np.sum(label_pred) #? => GT_labels - 4 events (M1-7)
            # AD_events_aggregated = len(event_starts) # gt_labels-few ?

            print(f"Model {Mod_i+1} is {model.name}")
            print("AD_events_aggregated in pps2 => ", AD_events_aggregated)
            print("Start of aggregated AD events:", event_starts)
            print("Ends  of aggregated AD events:", event_ends)


            # Ensure y_test is in the correct format (convert to list if necessary)
            from sklearn.preprocessing import MultiLabelBinarizer
            y_test_list = y_test.tolist() if isinstance(y_test, np.ndarray) else y_test

            # Convert true labels to binary format
            mlb = MultiLabelBinarizer()
            try:
                y_test_binary = mlb.fit_transform(y_test_list)
            except TypeError:
                # If TypeError occurs, y_test_list may contain float values
                y_test_binary = np.array(y_test_list).reshape(-1, 1)

            # Assuming rec_err is a numpy array
            min_rec_err = np.min(rec_err)
            max_rec_err = np.max(rec_err)
            num_thresholds = 5  # adjust me...

            # Create thresholds within the range of rec_err
            thresholds = np.linspace(min_rec_err, max_rec_err, num=num_thresholds)

            # Use y_test_binary instead of y_test
            for i, threshold in enumerate(thresholds):
                label_pred = [1 if x >= threshold else 0 for x in rec_err]
                label_pred, _, _ = adjust_pred(y_test_binary[:len(label_pred)], label_pred)
                p[i], r[i], f1[i], _ = precision_recall_fscore_support(
                    y_test_binary[:len(label_pred)], label_pred, average="binary", zero_division=0
                )

            def calculate_individual_kpi_means(data):
                """Calculates the mean of each KPI across all data."""
                return np.mean(data, axis=(0, 1))

            def calculate_total_error_in_window(data, reconstructions):
                """Calculates the total error in each window of data."""
                return np.sum(np.square(data - reconstructions), axis=(1, 2))

            def calculate_kpi_contributions_per_event(model, data):
                """Calculates MSE and KPI contributions within an event window."""
                # Ensure data is not empty
                if data.shape[1] > 0:
                    reconstructions = model.predict(data)
                    mse = np.mean(np.square(data - reconstructions), axis=(1, 2))
                    kpi_contributions = np.mean(np.abs(data - reconstructions), axis=(0, 1))
                else:
                    print(f"Invalid event window: data is empty")
                    mse = None
                    kpi_contributions = None

                return mse, kpi_contributions

            def accumulate_kpi_contributions_normalized(kpi_contributions_list, event_durations, individual_kpi_means, total_error_in_window):
                """Accumulates and normalizes KPI contributions per event."""
                accumulated_contributions = np.zeros_like(kpi_contributions_list[0])

                for i, (event_contributions, event_duration) in enumerate(zip(kpi_contributions_list, event_durations)):
                    normalized_contributions = event_contributions / (individual_kpi_means[i] or total_error_in_window[i])
                    accumulated_contributions += normalized_contributions[:event_duration]

                    # Logic for identifying and printing top contributing KPIs per event
                    contributing_kpis = np.where(normalized_contributions > 0)[0] + 1  # Adding 1 to convert from zero-based to one-based index
                    print(f"Event {i + 1} contributing KPIs:", contributing_kpis)
                    for kpi_index in contributing_kpis:
                        print(f"  KPI {kpi_index} contribution: {normalized_contributions[kpi_index - 1]}")

                print("Accumulated contributions for each KPI:", accumulated_contributions)
                return accumulated_contributions

                # Calculate the individual KPI means and total error in window
                individual_kpi_means = calculate_individual_kpi_means(x_test_reshaped)
                total_error_in_window = calculate_total_error_in_window(x_test_reshaped, model.predict(x_test_reshaped))

                # Calculate the sequence index and offset for each event start and end
                event_starts_seq_idx = [start // desired_sequence_length for start in event_starts]
                event_starts_offset = [start % desired_sequence_length for start in event_starts]
                event_ends_seq_idx = [end // desired_sequence_length for end in event_ends]
                event_ends_offset = [end % desired_sequence_length for end in event_ends]
                print(f"Number of events: {len(event_starts)}")

                # Iterate over aggregated events and calculate/accumulate KPI contributions
                for i in range(len(event_starts)):
                    start_seq_idx, start_offset = event_starts_seq_idx[i], event_starts_offset[i]
                    end_seq_idx, end_offset = event_ends_seq_idx[i], event_ends_offset[i]
                    print(f"Processing event {i+1}")

                    # Extract the sequences and offsets for the current event
                    event_sequences = x_test_reshaped[start_seq_idx:end_seq_idx+1]
                    if start_seq_idx == end_seq_idx:
                        # If the event is within a single sequence, slice it out
                        event_data = event_sequences[:, start_offset:end_offset, :]
                    else:
                        # If the event spans multiple sequences, slice out the first and last sequences and concatenate
                        first_sequence = event_sequences[0, start_offset:, :]
                        last_sequence = event_sequences[-1, :end_offset, :]
                        middle_sequences = event_sequences[1:-1] if len(event_sequences) > 2 else np.array([])
                        if middle_sequences.size > 0:
                            # Flatten middle_sequences along the sequence dimension
                            middle_sequences = middle_sequences.reshape(-1, middle_sequences.shape[-1])
                        event_data = np.concatenate([first_sequence, middle_sequences, last_sequence], axis=0) if middle_sequences.size > 0 else np.concatenate([first_sequence, last_sequence], axis=0)

                    # Ensure event_data has a 3D shape
                    if len(event_data.shape) < 3:
                        event_data = np.expand_dims(event_data, axis=0)

                    mse, kpi_contributions = calculate_kpi_contributions_per_event(model, event_data)
                    if kpi_contributions is not None:
                        # Additional logic for normalizing and accumulating KPI contributions
                        normalized_contributions = kpi_contributions / (individual_kpi_means[i] or total_error_in_window[i])
                        if i == 0:
                            # Initialize accumulated_contributions with the correct shape
                            accumulated_contributions = np.zeros_like(normalized_contributions)
                        accumulated_contributions += normalized_contributions
                        # Access the top contributing KPIs per AD event
                        top_contributors = np.argsort(accumulated_contributions)[::-1][:10]
                        print(f"Event {i+1}: Top-k contributing KPIs={top_contributors}")
                    else:
                        print(f"KPI contributions could not be calculated for event {i+1}")


            # Plot test labels and refined predicted events
            #machine_name = dataset
            label_file_path = f"/kaggle/working/ServerMachineDataset/processed/{machine_name}_test_label.pkl"
            gt_labels = pd.read_pickle(label_file_path)
            print(f"Model {Mod_i+1} is {model.name}")

            plt.figure(figsize=(22, 4))
            plt.plot(label_pred, alpha=0.7, label="Refined Predicted AD Events")
            plt.plot(gt_labels, alpha=0.9, label="Test Labels")
            plt.title("Test Labels (orange) vs Refined Predicted AD Events (blue) ")
            plt.xlabel("Time Step")
            plt.ylabel("Label")
            #plt.show()
            plt.savefig(os.path.join(machine_model_dir, f'Refined_AD_vs_GT_model_{Mod_i+1}.png'))
            plt.close()

            # Plot model's rec_err with a logarithmic scale on the Y-axis
            rec_err_normal = [x for i, x in enumerate(rec_err) if label_pred[i] == 0]
            rec_err_anomaly = [x for i, x in enumerate(rec_err) if label_pred[i] == 1]

            plt.figure(figsize=(8, 4))
            plt.hist(rec_err_normal, bins=250, color='blue', alpha=0.8)
            plt.hist(rec_err_anomaly, bins=250, color='red', alpha=1)
            plt.yscale('log')
            plt.xlabel('Reconstruction Error')
            plt.ylabel('Frequency (log scale)')
            plt.title('Histogram of AE Model\'s Processed Reconstruction Errors')
            plt.grid()
            #plt.show()
            plt.savefig(os.path.join(machine_model_dir, f'Histogram_recErr_model_{Mod_i+1}.png'))
            plt.close()


            def calculate_metrics_with_thresholds(rec_err, y_test, F_score_thresholds):
                """
                Calculate precision, recall, and F1 score for different thresholds based on standard deviations.

                Parameters:
                - rec_err (numpy array): Reconstruction errors.
                - y_test (numpy array or list): Ground truth labels (binary time series).
                - F_score_thresholds (list): List of standard deviation multipliers for threshold calculation.

                Returns:
                - metrics (numpy array): Array containing precision, recall, and F1 score for each threshold.
                """
                # Ensure y_test is in the correct format (convert to list if necessary)
                y_test_list = y_test.tolist() if isinstance(y_test, np.ndarray) else y_test

                # Assuming rec_err is a numpy array
                min_rec_err = np.min(rec_err)
                max_rec_err = np.max(rec_err)

                # Create thresholds within the range of rec_err
                thresholds = np.linspace(min_rec_err, max_rec_err, num=len(F_score_thresholds))

                # Initialize arrays to store metrics
                num_thresholds = len(thresholds)
                metrics = np.zeros((num_thresholds, 3))  # 3 columns for precision, recall, F1 score

                # Use y_test directly (no need for MultiLabelBinarizer)
                for i, threshold in enumerate(thresholds):
                    label_pred = [1 if x >= threshold else 0 for x in rec_err]
                    label_pred, _, _ = adjust_pred(y_test[:len(label_pred)], label_pred)
                    p, r, f1, _ = precision_recall_fscore_support(
                        y_test[:len(label_pred)], label_pred, average="binary", zero_division=0
                    )
                    metrics[i] = [p, r, f1]

                return metrics

            # Example usage:
            F_score_thresholds = [2, 3, 4, 5, 6, 7]
            metrics_result = calculate_metrics_with_thresholds(rec_err, y_test, F_score_thresholds)
            print("F_score funct.", metrics_result)


            # Recalculate F1, Precision, and Recall for different cutoffs based on simple std. deviations
            F_score_thshld = [2, 3, 4, 5, 6, 7]
            f1 = np.zeros(len(F_score_thshld))
            p = np.zeros(len(F_score_thshld))
            r = np.zeros(len(F_score_thshld))
            err_std = np.std(rec_err)

            for i, k in enumerate(F_score_thshld):
                t = np.mean(rec_err) + k * err_std
                label_pred = [1 if x >= t else 0 for x in rec_err]
                label_pred, _, _ = adjust_pred(y_test_binary[:len(label_pred)], label_pred)
                p[i], r[i], f1[i], _ = precision_recall_fscore_support(
                    y_test_binary[:len(label_pred)], label_pred, average="binary", zero_division=0
                )

                from sklearn.preprocessing import MultiLabelBinarizer
                from sklearn.metrics import precision_recall_fscore_support


            # Print scores before and after AD event aggregation
            print(f"Model {Mod_i+1} is {model.name}")
            print("Scores before AD event aggregation")
            print('Prec=', P_bef, 'Rec=', R_bef, 'F1=', F1_bef)
            print("========================================================================= \n")
            print("Scores after AD event aggregation and k-sigma = [2,3,4,5,6,7,8] thresholding")
            print("Prec=", p)
            print("Rec=", r)
            print("F1=", f1)

            # Plot precision, recall, and F1 vs. thresholds
            print(f"Model {Mod_i+1} is {model.name}")
            plt.figure(figsize=(10, 4))
            plt.plot(F_score_thshld, p, marker='o', linestyle='-', label='Precision')
            plt.plot(F_score_thshld, r, marker='o', linestyle='-', label='Recall')
            plt.plot(F_score_thshld, f1, marker='o', linestyle='-', label='F1')
            plt.xlabel('Thresholds')
            plt.ylabel('Score')
            plt.title('Precision, Recall, and F1 vs. Thresholds')
            plt.legend()
            plt.grid()
            #plt.show()
            plt.savefig(os.path.join(machine_model_dir, f'F1_scores_{Mod_i+1}.png'))
            plt.close()

            # Print F1 scores
            # print(f1)

            # Append and save the AD prediction performance metrics per machine and model
            performance_metrics.append({
                'P_bef': P_bef,
                'R_bef': R_bef,
                'F1_bef': F1_bef,
                'p': p,
                'r': r,
                'f1': f1
            })

            # Save performance metrics to a pickle file for internal use; later we'll save the aggregates into one CSV file
            with open(os.path.join(machine_model_dir, f'performance_metrics_model_{Mod_i+1}.pkl'), 'wb') as f:
                pickle.dump(performance_metrics[-1], f)

            #=========================================================================================
            # KPI Visualisations for train & validation reconstructions vs. the original resp. KPIs
            # Take one batch from the training dataset
            batch = [batch[0] for i, batch in enumerate(ds_train) if i == 0]

            # Check the shape of the selected batch
            np.shape(batch)  # This should display (1, 1024, 120, 38)

            # Predict using the model and the selected batch
            batch_pred = model.predict(batch[0])

            # Check the shape of the prediction
            print(batch_pred.shape)

            # Subsample every 120 time steps in the second dimension, reducing it from 1024 to 9
            batch_pred = batch_pred[::120, :, :]

            # Check the shape after subsampling
            print(batch_pred.shape)

            # Combine all 9 blocks of data into a single long array with 38 features for each of the 1080 samples
            batch_pred = batch_pred.reshape((-1, 38))

            # Check the final shape
            print(batch_pred.shape)

            # Calculate the new size by finding how many complete groups of 38 elements fit within the original size
            curr_pred_shape = np.shape(pred)
            new_size = ( curr_pred_shape[0] // 38) * 38
            pred = pred[:new_size]

            # Reshape the tensor to have 38 columns while inferring the number of rows to maintain the total number of elements
            pred = tf.reshape(pred, (-1, 38))

            print(pred)
            print(f"Model {Mod_i+1} is {model.name}")

            # KPIs train & validation reconstructions (): Combined plots of IN vs. OUT/predicted KPIs
            plt.rcParams["figure.figsize"] = (30, 30)

            fig, ax = plt.subplots(38, 1)
            gt_data = np.array(batch[0][::120, :, :]).reshape((-1, 38))

            for i in range(38):
                ax[i].plot(gt_data[:1000, i], label="IN", color="red")
                ax[i].plot(batch_pred[:1000, i], label="Pred", color="blue")
                ax[i].set_ylabel(f"TKPI {i+1}")
                ax[i].legend()

            #plt.show()
            plt.savefig(os.path.join(machine_model_dir, f'KPIs_train_model_{Mod_i+1}.png'))
            plt.close()
            print(f"Model {Mod_i+1} is {model.name}")

            # Take one batch from the test/validation dataset
            batch = [batch[0] for i, batch in enumerate(ds_validation) if i == 0]

            # Check the shape of the selected batch
            np.shape(batch)  # This should display (1, 1024, 120, 38)

            # Predict using the model and the selected batch
            batch_pred = model.predict(batch[0])

            # Check the shape of the prediction
            print(batch_pred.shape)

            # Subsample every 120 time steps in the second dimension, reducing it from 1024 to 9
            batch_pred = batch_pred[::120, :, :]

            # Check the shape after subsampling
            print(batch_pred.shape)

            # Combine all 9 blocks of data into a single long array with 38 features for each of the 1080 samples
            batch_pred = batch_pred.reshape((-1, 38))

            # Check the final shape
            print(batch_pred.shape)
            print(f"Model {Mod_i+1} is {model.name}")

            # Calculate the new size by finding how many complete groups of 38 elements fit within the original size
            curr_pred_shape = np.shape(pred)
            new_size = ( curr_pred_shape[0] // 38) * 38
            pred = pred[:new_size]

            # Reshape the tensor to have 38 columns while inferring the number of rows to maintain the total number of elements
            pred = tf.reshape(pred, (-1, 38))

            print(pred)
            print(f"Model {Mod_i+1} is {model.name}")

            # KPIs train & validation reconstructions (): Combined plots of IN vs. OUT/predicted KPIs
            plt.rcParams["figure.figsize"] = (30, 30)

            fig, ax = plt.subplots(38, 1)
            gt_data = np.array(batch[0][::120, :, :]).reshape((-1, 38))

            for i in range(38):
                ax[i].plot(gt_data[:1000, i], label="IN", color="red")
                ax[i].plot(batch_pred[:1000, i], label="Pred", color="blue")
                ax[i].set_ylabel(f"VKPI {i+1}")
                ax[i].legend()

            #plt.showvalid
            plt.savefig(os.path.join(machine_model_dir, f'KPIs_valid_model_{Mod_i+1}.png'))
            plt.close()
            print(f"Model {Mod_i+1} is {model.name}")


load data of: machine-1-7
train:  0 None
test:  0 None
train set shape:  (23697, 38)
test set shape:  (23697, 38)
test set label shape:  (23697,)
Epoch 1/25
24/24 [==============================] - 3s 106ms/step - loss: 0.0070 - mse: 0.0070 - mae: 0.0470 - val_loss: 0.0057 - val_mse: 0.0057 - val_mae: 0.0431 - lr: 0.1000
Epoch 2/25
24/24 [==============================] - 2s 99ms/step - loss: 0.0079 - mse: 0.0079 - mae: 0.0493 - val_loss: 0.0099 - val_mse: 0.0099 - val_mae: 0.0582 - lr: 0.1000
Epoch 3/25
24/24 [==============================] - 2s 98ms/step - loss: 0.0071 - mse: 0.0071 - mae: 0.0469 - val_loss: 0.0100 - val_mse: 0.0100 - val_mae: 0.0603 - lr: 0.1000
Epoch 4/25
24/24 [==============================] - 2s 101ms/step - loss: 0.0071 - mse: 0.0071 - mae: 0.0465 - val_loss: 0.0091 - val_mse: 0.0091 - val_mae: 0.0571 - lr: 0.1000
Epoch 5/25
24/24 [==============================] - 2s 96ms/step - loss: 0.0072 - mse: 0.0072 - mae: 0.0475 - val_loss: 0.0060 - val_mse: 0.0060 - v

In [ ]:
stop


In [ ]:
# Save all (perf. only, since training history was saved during training) metrics into a .CSV file
# import pandas as pd
# import os
# import pickle

# # Main loop#1: All or selected subsets of the 28 machine names from the SMD dataset
# machine_names = ["machine-2-4", "machine-2-5", "machine-2-6", "machine-2-7", "machine-2-8", "machine-2-9", "machine-3-1", "machine-3-2", "machine-3-3", "machine-3-4", "machine-3-5", "machine-3-6", "machine-3-7", "machine-3-8", "machine-3-9", "machine-3-10", "machine-3-11"]
# # List of the 4 AE model names
# model_names = ["snu-ae", "snu-transformer", "lstm", "transformer"]

# # Define a dictionary for model indices
# model_indices = {
#     "snu-ae": 1,
#     "snu-transformer": 2,
#     "lstm": 3,
#     "transformer": 4
# }

# Initialize a list to store the models again, iff we build and append all the models anew (rare need)
# model_list = []

# Initialize an empty list to store all performance metrics
all_performance_metrics = []

# Loop over each machine
for i_mach, machine_name in enumerate(machine_names):
    # Define the base directory
    base_dir = os.path.join(results_dir, machine_name)

    # Loop over each model
    for model_name_str, model_index in model_indices.items():
        # Check if the pickle file exists
        pickle_filepath = os.path.join(base_dir, f'{model_name_str}/performance_metrics_model_{model_index}.pkl')
        exists = os.path.isfile(pickle_filepath)

        # If the file exists, load the performance metrics
        if exists:
            with open(pickle_filepath, 'rb') as f:
                performance_metrics = pickle.load(f)

            # Add the machine name and model name to the performance metrics
            performance_metrics['machine_name'] = machine_name
            performance_metrics['model_name'] = model_name_str

            # Add the performance metrics to the list
            all_performance_metrics.append(performance_metrics)
        else:
            # If the file doesn't exist, set the performance metrics to -1
            performance_metrics = {'machine_name': machine_name, 'model_name': model_name_str, 'avg_score': -1}
            all_performance_metrics.append(performance_metrics)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_performance_metrics)

# Save the DataFrame to a CSV file
df.to_csv(os.path.join(results_dir, 'performance_metrics.csv'), index=False)


In [ ]:
# save only the max F1 values (incl. its Prec and Recall)

# Initialize an empty list to store all performance metrics
all_performance_metrics = []

# Loop over each machine
for i_mach, machine_name in enumerate(machine_names):
    # Define the base directory
    base_dir = os.path.join(results_dir, machine_name)

    # Loop over each model
    for model_name_str, model_index in model_indices.items():
        # Check if the pickle file exists
        pickle_filepath = os.path.join(base_dir, f'{model_name_str}/performance_metrics_model_{model_index}.pkl')
        exists = os.path.isfile(pickle_filepath)

        # If the file exists, load the performance metrics
        if exists:
            with open(pickle_filepath, 'rb') as f:
                performance_metrics = pickle.load(f)

            # Find the index of the maximum F1 score
            max_f1_index = np.argmax(performance_metrics['f1'])

            # Use this index to find the corresponding precision and recall
            p_max = performance_metrics['p'][max_f1_index]
            r_max = performance_metrics['r'][max_f1_index]
            f1_max = performance_metrics['f1'][max_f1_index]

            # For global CSV: Add the machine name, model name, and maximum scores to the performance metrics
            performance_metrics_max = {
                'machine_name': machine_name,
                'model_name': model_name_str,
                'p_max': p_max,
                'r_max': r_max,
                'f1_max': f1_max,
                'max_f1_index': max_f1_index
            }

            # Add the performance metrics to the list
            all_performance_metrics.append(performance_metrics_max)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_performance_metrics)

# Save the DataFrame to a CSV file
df.to_csv(os.path.join(results_dir, 'performance_metrics_max.csv'), index=False)

In [ ]:
# Plot the final (best) F1 scores of all 4 models across all 28 SMD machines

import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np

# machine_names = ["machine-1-1", "machine-1-2", "machine-1-3", "machine-1-4", "machine-1-5", "machine-1-6", "machine-1-7", "machine-1-8", "machine-2-1", "machine-2-2", "machine-2-3"]#, "machine-2-4", "machine-2-5", "machine-2-6", "machine-2-7", "machine-2-8", "machine-2-9", "machine-3-1", "machine-3-2", "machine-3-3", "machine-3-4", "machine-3-5", "machine-3-6", "machine-3-7", "machine-3-8", "machine-3-9", "machine-3-10", "machine-3-11"]
# # List of the 4 AE model names
# model_names = ["snu-ae", "snu-transformer", "lstm", "transformer"]

#results_dir = 'C:\\Downloads\\smd\\res1\\kaggle\\working\\results\\'

# Define the model colors
model_colors = {
    'snu-ae': 'cyan',
    'snu-transformer': 'blue',
    'lstm': 'red',
    'transformer': 'black'
}

# Load the DataFrame from the CSV file
df = pd.read_csv(os.path.join(results_dir, 'performance_metrics_max.csv'))

# Loop over each model
for model_name in model_names:
    # Filter the DataFrame for the current model
    df_model = df[df['model_name'] == model_name]

    # Plot the maximum F1 score for the current model
    plt.plot(df_model['machine_name'], df_model['f1_max'], color=model_colors[model_name], label=model_name)


    # Add annotations for the max F1 indices (K-sigma cut-off thresholds,for our AD a value within 2-8 std. deviations)
    for i, max_f1_index in enumerate(df_model['max_f1_index']):
        plt.annotate(str(max_f1_index+1), (df_model['machine_name'].iloc[i], df_model['f1_max'].iloc[i]))


# Add labels and a legend to the plot
plt.xlabel('Machine')
plt.ylabel('Max F1 Score')
plt.title('Max F1 Score for Each Model Across All Machines')
plt.legend()

# # Display the plot
# plt.show()

plt.savefig(os.path.join(results_dir, f'F1_all_models.png'))
plt.close()


In [ ]:
print("Working dirs at FINALE =>", results_dir, "base_dir=", base_dir, "machine_name=", machine_name, "model_name_str=", model_name_str, "machine_model_dir=", machine_model_dir)
# !zip -r smd_res2.zip /kaggle/working/results2/
# !rm -r /kaggle/working/results2

## Prediction